<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/Google_Podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong>Google Podcast 爬蟲(用於Colab平台)</strong></h1>

<p>爬取Google Podcast，只要輸入頻道的網址，便能擷取所有的單集，並包含以下資訊：出刊日、標題、長度、單集連結、音訊連結、單集摘要</p>

<p><strong>可同時下載各集音檔</strong></p>


# 頻道網址設定

In [179]:
#url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9hbmNob3IuZm0vcy8yMDk1NDFiOC9wb2RjYXN0L3Jzcw?sa=X&ved=0CAYQrrcFahcKEwigwavVo5H3AhUAAAAAHQAAAAAQAQ' #財報狗
#url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkcy5zb3VuZG9uLmZtL3BvZGNhc3RzLzk1NDY4OWE1LTMwOTYtNDNhNC1hODBiLTc4MTBiMjE5Y2VmMy54bWw' #股癌
#url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9hcGkuc291bmRvbi5mbS92Mi9wb2RjYXN0cy9jOTJjYjY2YS0xNGZjLTQxMmYtODRhYS0wODEyNWFkZDVjYTkvZmVlZC54bWw?sa=X&ved=0CAMQ4aUDahcKEwiYgunN8JL3AhUAAAAAHQAAAAAQLQ' #IEO 看什麼
url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9hcGkuc291bmRvbi5mbS92Mi9wb2RjYXN0cy80YTg2NjBhMC1lMGQwLTQ5MGItOGQ0Ni1jMjgyMTk2MDZmNDcvZmVlZC54bWw' #美股投資學－財女Jenny
#url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9hcGkuc291bmRvbi5mbS92Mi9wb2RjYXN0cy9kMzkwNzk2MS1lYjI0LTQ3MDUtOGVmMS01MTdlY2JkZmJjNDYvZmVlZC54bWw' #Gamma - 科技投資




# 安裝並載入套件

In [45]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
#!apt-get install xvfb
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains


!pip install fake_useragent
from fake_useragent import UserAgent
from lxml import etree
import requests
import pandas as pd
import random
import time

from google.colab import drive
drive.mount('/content/drive')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graph

#瀏覽器參數設定

In [108]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


#擷取頻道資料

In [180]:
headers = UserAgent().random
req = requests.get(url, headers={"User-Agent":headers})
req = req.content.decode()
req = etree.HTML(req)

channel_name = req.xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[1]/div[1]/div[1]/text()')
channel_name = str(channel_name[0])
date = req.xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[2]/div[2]/a/div/div[1]/text()')
title = req.xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[2]/div[2]/a/div/div[2]/div/text()')
length = req.xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[2]/div[2]/a/div/div[4]/div[1]/div/div/span/text()')
links = req.xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[2]/div[2]/a/@href')
Summary = req.xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[2]/div[2]/a/div/div[3]/text()')

for i in range(len(title)):
  url = links[i]
  url = url[1:len(url)]
  links[i] = 'https://podcasts.google.com' + url
  Summary[i] = Summary[i].replace('\n', '').replace('\xa0', '')

date = pd.DataFrame(date)
title = pd.DataFrame(title)
length = pd.DataFrame(length)
links = pd.DataFrame(links)
Summary = pd.DataFrame(Summary)
temp_data = pd.concat([date, title, length, links], axis=1)
temp_data.columns = ['date', 'title', 'length', 'link']

In [181]:
#擷取音訊連結
audio_link = []

for i in range(len(temp_data)):
  url = temp_data['link'][i]
  browser.get(url)

  t = random.uniform(1, 1.5) #隨機數
  time.sleep(t) #暫停

  element = browser.find_element(by=By.XPATH, value='//*[@id="yDmH0d"]/c-wiz/div/div[2]/div/div[1]/div[3]/div[1]')
  temp_audio = element.get_attribute("jsdata").replace('Kwyn5e;', '').replace(';2', '')
  audio_link.append(temp_audio)

audio_link = pd.DataFrame(audio_link)

#整理與檢視資料



In [182]:
data = pd.concat([date, title, length, links, audio_link, Summary], axis=1)
data.columns = ['date', 'title', 'length', 'link', 'audio_link', 'Summary']
data

,date,title,length,link,audio_link,Summary
0,3 days ago,EP123 人生理財、投資策略，個股資訊Q&A大平台,35 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,🔒 訂閱Pressplay專欄『美股投資學』最新文章https://pse.is/43ae2...
1,"Apr 7, 2022",EP122 財女說書《選擇，不只是選擇》：反向思考，避開投資陷阱,24 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,🔒 訂閱Pressplay專欄『美股投資學』最新文章https://pse.is/43ae2...
2,"Apr 4, 2022",EP121 Tesla第一季交付年增70%優於預期，如何找到下一個Tesla？,35 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,🔒 訂閱Pressplay專欄『美股投資學』最新文章https://pse.is/43ae2...
3,"Mar 31, 2022",EP120 Nvidia GTC大會，日常作息如何提高生產力?,39 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,保養跟投資一樣，選對商品，長期持有！好皮膚需要時間照顧，但堅持一定可以看到成效推薦給大家法國...
4,"Mar 28, 2022","EP119 Tesla, Apple股價轉強帶動美股大盤上攻～巴菲特投資組合為什麼強？",29 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,🔒 訂閱Pressplay專欄『美股投資學』最新文章https://pse.is/43ae2...
...,...,...,...,...,...,...
118,"Jan 28, 2021",EP05 美股大幅拉回該緊張嗎？介紹一家已經賺錢的SaaS成長股Servicenow(NOW),13 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,美股科技巨頭財報利多不漲，拉回是否可以進場布局？今天介紹一家也剛公布優異財報的SaaS成長股...
119,"Jan 26, 2021",EP04 從GameStop(GME)之亂，學習散戶的多空應變之道,17 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,機構與散戶之戰尚未結束，周一開盤GME股價一度飆漲至144%，至收盤收斂至18%散戶到底該追...
120,"Jan 18, 2021",EP03 Howard Marks備忘錄-美股中價值與成長的差異，成長股該如何投資？,17 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,每一次的危機都是一次財富重分配的機會，在極度悲觀的市場入市的投資人，享受資產重新上漲的獲利，...
121,"Jan 10, 2021",EP02 亞馬遜、摩根大通與波克夏合資的醫保公司Haven結束，美股遠距醫療產業起飛！,13 min,https://podcasts.google.com/feed/aHR0cHM6Ly9hc...,https://chtbl.com/track/FB36A5/rss.soundon.fm/...,亞馬遜、摩根大通與波克夏合資的醫保公司Heaven結束，聯合健康保險與CVS股價沒有反應，卻...


#匯出資料

In [183]:
#輸出檔案
path = '/content/drive/MyDrive/Data Set/Podcast/'+channel_name+'.csv' #路徑
data.to_csv(path, index=None)

#下載Podcast音檔

In [166]:
#判別檔案格式
if 'mp3' in data['audio_link'][1]:
  file_type = '.mp3'
else:
  file_type = '.m4a'

#for i in range(3):
for i in range(len(data['link'])):
  url = data['audio_link'][i]

  #設定檔名
  audio_name = channel_name + ' ' + data['date'][i]
  downloaded_file_location = '/content/drive/MyDrive/Data Set/Podcast/'+ audio_name + file_type

  # Downloading the audio file
  headers = UserAgent().random  #隨機請求表頭
  req = requests.get(url, headers={"User-Agent":headers}).content

  t = random.uniform(0.5, 1) #隨機數
  time.sleep(t) #暫停

  with open(downloaded_file_location, 'wb') as f:
      f.write(req)